### Import modules and things

In [130]:
from pymongo import MongoClient
import pprint
import json
import bs4
from bs4 import BeautifulSoup
import re
import numpy as np

### Get mongoclient

In [3]:
client = MongoClient('localhost', 27017)
db=client['reviews']
coll=db['book_reviews']

In [9]:
documents = [x for x in db['book_reviews'].find()]

In [36]:
test = documents[500]
# test is a mongo document, not the html. we need to get the 'reviews' value
# test['reviews']


In [37]:
testsoup = BeautifulSoup(test['reviews'], 'html.parser')

In [45]:
text = testsoup.find_all(class_ = "readable")

### Okay, we got to "text", sort of. Lets try to clean this up. 

In [95]:
type(text)

bs4.element.ResultSet

In [80]:
def clean_readable(readable):
    '''
    a function that takes readable, a bs4 ResultSet and turns it into a list of the strings contained within. 
    '''
    
    

In [103]:
type(text[0].contents)

list

In [105]:
l = text[0].contents

In [113]:
l[0]

'\n'

In [117]:
type(l[3])

bs4.element.Tag

In [124]:
lc = [i for i in l[3].children]

In [190]:
#display different tags in the result. 


# for i in lc:
#     #print(type(i))
#     if type(i) == bs4.element.Tag:
#         print(i)
#     #if type(i) == bs4.element.NavigableString:
# #     print(i)

# get rid of all these '< br/ >', 
they're line breaks. maybe divide lines up by line breaks? no. DO NOT. that will make the nlp treat each line differently, we want an nlp document to be a whole review

In [195]:
def remove_text_inside_brackets(text, brackets="<>"):
    count = [0] * (len(brackets) // 2) # count open/close brackets
    saved_chars = []
    for character in text:
        for i, b in enumerate(brackets):
            if character == b: # found bracket
                kind, is_close = divmod(i, 2)
                count[kind] += (-1)**is_close # `+1`: open, `-1`: close
                if count[kind] < 0: # unbalanced bracket
                    count[kind] = 0  # keep it
                else:  # found bracket to remove
                    break
        else: # character is not a [balanced] bracket
            if not any(count): # outside brackets
                saved_chars.append(character)
    return ''.join(saved_chars)

In [189]:
#ideally, replace </b> with ! to take advantage of vader tech emphasis tech

smalltext = '''This was initially going to be four stars, but the last 100/150 pages were just so 
<b>un-deniably</b>
 amazing I couldn’t help but bump it up half a star. For my first few reads they’ve started off quite light, however within the first chapter this was 
<u>dark</u>
! We got deaths and treachery, and I was 
<b>here for it/</b>'''
re.replace(r'</b>','!',smalltext)

    

In [185]:
slc = [str(i) for i in lc]

In [192]:
len(text)

1

In [193]:
text = testsoup.find_all(class_ = "readable")
l = text[0].contents # possible breakpoint, what is at the other text locations?
lchildren = [i for i in l[3].children]

strlchildren = [str(i) for i in lchildren]
seperator = ' '
bigstring = seperator.join(strlchildren)
review_nlp_document = remove_text_inside_brackets(bigstring)

In [194]:
print('text holds this many items: '+ str(len(text)))

text holds this many items: 1


In [198]:
test = documents[498:500]

##make this thing create a whole bunch of things in a loop from documents
all_reviews = []
for i in test:
    breakpoint = documents.index(i) ##will show me where this thing breaks
    soup = BeautifulSoup(i['reviews'], 'html.parser')
    text = soup.find_all(class_ = "readable")
    l = text[0].contents # possible breakpoint, what is at the other text locations?
    lchildren = [i for i in l[3].children]

    strlchildren = [str(i) for i in lchildren]
    seperator = ' '
    bigstring = seperator.join(strlchildren)
    review_nlp_document = remove_text_inside_brackets(bigstring)
    all_reviews.append(review_nlp_document)

IndexError: list index out of range

In [200]:
text

[<span class="readable" id="reviewTextContainer388324260">
 <span id="freeTextContainer12062068264074714425">Harry Potter changed my life. If I hadn’t read Harry Potter when I did I would not be the same person today. What I am today, in a part is due to Harry. I cannot really say enough to make you understand how much Harry Potter means to me.</span>
 </span>]

In [201]:
!pwd


/home/robert/Assignments/Predictotron/src
